In [26]:
import numpy as np
import random
import pandas as pd
from tqdm import tqdm
from scipy import stats
import matplotlib.pyplot as plt
from nilearn import datasets
from nilearn.maskers import NiftiLabelsMasker 
import joblib
from pathlib import Path

In [6]:
# load dataset containing sex and participant id
# df = pd.read_csv("/home/xlajoie/Desktop/Final_HCP_database.csv")
df = pd.read_csv("/data/brambati/cbedetti/sandbox/xanthy/Final_HCP_database.csv")

# Classification et trouver seed region qui donnent une meilleure classification

In [7]:
def img4d2vector(img_path, masker):
    img_masked = masker.fit_transform(img_path)  #fait une moyenne par label 
    return img_masked.flatten()  #devient 1 vecteur

def vector2img4d(vector, masker):
    data_2d = vector.reshape(8, -1) # 8
    return masker.inverse_transform(data_2d)   #remettre dans espace MNI that we can plot with nilearn

In [8]:
def bootstrap_scores(boot_coefs):
    """
    Calculate z scores and p-value based on bootstrap coefficients
    
    Parameters
    ----------
    boot_coefs: bootstrap coefficients (array-like)
    
    Returns
    ----------
    z_scores: z scores calculated from bootstrap coefficients
    pval: p-value calculated from z-scores
    pval_bonf: corrected p-values using bonferonni correction
    
    See also nltools summarized_bootstrap function: https://nltools.org/_modules/nltools/stats.html
    """
    mean_scores = np.mean(boot_coefs, axis=0)
    z_scores = np.mean(boot_coefs, axis=0)/np.std(boot_coefs, axis=0)
    assert np.sum(np.isnan(z_scores)) == 0
    pval = 2 * (1 - stats.norm.cdf(np.abs(z_scores)))
    pval_bonf = np.where(pval < (0.05/len(pval)), pval, 0)
    #boot_z_fdr = np.where(pval < fdr(pval, q=0.05), pval, 0)
    
    return mean_scores, z_scores, pval, pval_bonf#, boot_z_fdr

In [20]:
def run(df_boot, data, seed):
    df_bootstrap = pd.DataFrame()
    for j in range(0, len(df_boot)):
        index = random.randint(0, len(df_boot)-1)
        frames = [df_bootstrap, df_boot[index:index+1]]
        df_bootstrap = pd.concat(frames)

    df_bootstrap = df_bootstrap.drop(df_bootstrap.columns[0], axis=1)


    # print(df_bootstrap["Gender"])  # should have the whole list

    x_correl = []
    nb_subjects = len(df_bootstrap)
    subject_label = df_bootstrap["subject_label"][:nb_subjects]

    x_correl = []
    for sub in tqdm(subject_label):
        for seed_name in seed:
            x_correl.append(data[sub, seed_name])

    x_correl = np.array(x_correl)
    x_correl = x_correl.reshape(len(df_boot), len(seed)*148)  # autant de lignes que de sujets, autant de colones (nb region atlas x 8 (seeds))


    y_sex = df_bootstrap["Gender"][:nb_subjects]  # maybe list(df["Gender"])

    # coef = machine_learning(x_correl, y_sex)

    return x_correl

In [10]:
def regionsOfInterest(accuracy, pvalue, moyenne, rois, label, x_correl):
    pvalue_val = pvalue.reshape(len(rois), 148).T
    pvalue_uncorrected = pvalue_val < 0.05

    data_pval_uncorrected = pd.DataFrame(pvalue_uncorrected, columns=rois)
    data_pval = pd.DataFrame(pvalue_val, columns=rois)

    pval_corrected = np.multiply(pvalue_val, pvalue_uncorrected)
    
    X_std = x_correl.std()
    weighted_coef = moyenne * X_std
    label_name = list(label['name']) * len(rois)
    seed_name = np.repeat(rois, 148)
    
    weighted_tab = pd.DataFrame(weighted_coef, columns=["coefficient"])
    weighted_tab['seed'] = seed_name
    weighted_tab['labels'] = label_name
    
    # print the 10 best seed anf their labels
    print(weighted_tab.sort_values(by=['coefficient'], ascending=False).head(10))
    
    
    # weighted_coef[weighted_coef<0.054609] = 0 # maybe changed the 0.054609 for a variable
        
    
    # printing the accuracy and its interval of confidance
    acc_test = np.asarray(accuracy)
    moy = np.mean(acc_test)
    se = np.std(acc_test)
    lower = moy - (1.96 * se)
    upper = moy + (1.96 * se)
    
    print("The mean accuracy : " + str(moy))
    print("The standard deviation of the accuracy : " + str(se))
    print("The lower boundry of the confidance interval of the accuracy : " + str(lower))
    print("The upper boundry of the confidance interval of the accuracy : " + str(upper))
    
    return weighted_tab

## Single run with all seed

In [12]:
SEEDS = ["opIFG_L", "planumtemp_L", "aMTG_L", "pITG_L", "opIFG_R", "planumtemp_R", "aMTG_R", "pITG_R"]
img_tpl = "/data/brambati/dataset/HCP/derivatives/seed-to-voxel-nilearn/results_3D/{seed_name}/sub-{participant_id}/sub-{participant_id}_seed-{seed_name}_mean-4_voxelcorrelations.nii.gz"
atlas_dest = datasets.fetch_atlas_destrieux_2009()
masker = NiftiLabelsMasker(atlas_dest.maps)

data = {}
for seed in SEEDS:
    for sub in tqdm(df["subject_label"]):
        img_path = img_tpl.format(seed_name=seed, participant_id=sub)
        data[(sub, seed)] = img4d2vector(img_path, masker)

/data/brambati/local/venvs/lingualab_workstation/lib/python3.8/site-packages/nilearn/datasets/atlas.py:288: UserWarning: `legacy_format` will default to `False` in release 0.11. Dataset fetchers will then return pandas dataframes by default instead of recarrays.
  warnings.warn(_LEGACY_FORMAT_MSG)
100%|█████████████████████████████████████████████████████████████████████████████████| 570/570 [01:28<00:00,  6.44it/s]


In [15]:
x_correl = []
for sub in tqdm(df["subject_label"]):
    line_temp = []
    for seed in SEEDS:
        line_temp.append(data[(sub, seed)])
    x_correl.append(np.concatenate(line_temp))
x_correl = np.array(x_correl)

100%|█████████████████████████████████████████████████████████████████████████████| 570/570 [00:00<00:00, 125196.55it/s]


In [18]:
x_correl.shape

(570, 1184)

## Bootstrap result all seed, single seed, all left seed and all right seed

In [23]:
study_dir = Path('/data/brambati/dataset/HCP/derivatives/training_sex_diff/')
results_path_all_seed = study_dir / "results" / "LinearSVC" / "models_iteration-10000_seeds-all_mean_LinearSVC.pkl"
# results_path_aMTG_L = study_dir / "results" / "LinearSVC" / "models_iteration-10000_seeds-aMTG_L_mean_LinearSVC.pkl"
# results_path_aMTG_R = study_dir / "results" / "LinearSVC" / "models_iteration-10000_seeds-aMTG_R_mean_LinearSVC.pkl"
# results_path_opIFG_L = study_dir / "results" / "LinearSVC" / "models_iteration-10000_seeds-opIFG_L_mean_LinearSVC.pkl"
# results_path_opIFG_R = study_dir / "results" / "LinearSVC" / "models_iteration-10000_seeds-opIFG_R_mean_LinearSVC.pkl"
# results_path_pITG_L = study_dir / "results" / "LinearSVC" / "models_iteration-10000_seeds-pITG_L_mean_LinearSVC.pkl"
# results_path_pITG_R = study_dir / "results" / "LinearSVC" / "models_iteration-10000_seeds-pITG_R_mean_LinearSVC.pkl"
# results_path_planumtemp_L = study_dir / "results" / "LinearSVC" / "models_iteration-10000_seeds-planumtemp_L_mean_LinearSVC.pkl"
# results_path_planumtemp_R = study_dir / "results" / "LinearSVC" / "models_iteration-10000_seeds-planumtemp_R_mean_LinearSVC.pkl"
# results_path_all_left = study_dir / "results" / "LinearSVC" / "models_iteration-10000_seeds-left_mean_LinearSVC.pkl"
# results_path_all_right = study_dir / "results" / "LinearSVC" / "models_iteration-10000_seeds-right_mean_LinearSVC.pkl"


results = joblib.load(results_path_all_seed)

coefs = []
for result in results:
    coefs.append(result["model"].coef_[0])

coefs = np.array(coefs)

data_results = pd.DataFrame.from_dict(results)
moyenne, scoresZ, pvalue, pvalue_bonf = bootstrap_scores(coefs)
moyenne.shape

/data/brambati/local/venvs/lingualab_workstation/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearSVC from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


(1184,)

In [24]:
atlas_dest = datasets.fetch_atlas_destrieux_2009(legacy_format=False)
label = atlas_dest["labels"].drop([0,42,117]) # correction for the destrieux atlas labels
label = label.reset_index()

## Best regions for all seeds

In [28]:
rois = ["opIFG_L", "planumtemp_L", "aMTG_L", "pITG_L", "opIFG_R", "planumtemp_R", "aMTG_R", "pITG_R"] # modified to rois of interest
run_1 = run(df, data, ["opIFG_L", "planumtemp_L", "aMTG_L", "pITG_L", "opIFG_R", "planumtemp_R", "aMTG_R", "pITG_R"])
ROIs = regionsOfInterest(data_results["acc_test"], pvalue, moyenne, rois, label, run_1)

100%|█████████████████████████████████████████████████████████████████████████████| 570/570 [00:00<00:00, 594714.75it/s]

      coefficient          seed                      labels
349      0.066980        aMTG_L               L S_front_sup
394      0.066608        aMTG_L      R G_pariet_inf-Angular
530      0.061722        pITG_L       R G_front_inf-Orbital
300      0.060541        aMTG_L  L G_and_S_transv_frontopol
794      0.059514  planumtemp_R      L S_interm_prim-Jensen
1056     0.059044        pITG_R     L G_oc-temp_lat-fusifor
453      0.057656        pITG_L     L G_cingul-Post-ventral
41       0.056609       opIFG_L            L Pole_occipital
783      0.055820  planumtemp_R               L S_calcarine
941      0.051546        aMTG_R               L S_front_sup
The mean accuracy : 0.916316814159292
The standard deviation of the accuracy : 0.027862591976324208
The lower boundry of the confidance interval of the accuracy : 0.8617061338856966
The upper boundry of the confidance interval of the accuracy : 0.9709274944328874


In [36]:
rois = ["opIFG_L", "planumtemp_L", "aMTG_L", "pITG_L", "opIFG_R", "planumtemp_R", "aMTG_R", "pITG_R"] # modified to rois of interest
run_1 = run(df, data, ["opIFG_L", "planumtemp_L", "aMTG_L", "pITG_L", "opIFG_R", "planumtemp_R", "aMTG_R", "pITG_R"])
ROIs = regionsOfInterest(data_results["acc_test"], pvalue, moyenne, rois, label, run_1)

100%|█████████████████████████████████████████████████████████████████████████████| 570/570 [00:00<00:00, 568820.67it/s]

      coefficient          seed                      labels
349      0.066486        aMTG_L               L S_front_sup
394      0.066117        aMTG_L      R G_pariet_inf-Angular
530      0.061267        pITG_L       R G_front_inf-Orbital
300      0.060094        aMTG_L  L G_and_S_transv_frontopol
794      0.059075  planumtemp_R      L S_interm_prim-Jensen
1056     0.058609        pITG_R     L G_oc-temp_lat-fusifor
453      0.057231        pITG_L     L G_cingul-Post-ventral
41       0.056191       opIFG_L            L Pole_occipital
783      0.055409  planumtemp_R               L S_calcarine
941      0.051165        aMTG_R               L S_front_sup
The mean accuracy : 0.916316814159292
The standard deviation of the accuracy : 0.027862591976324208
The lower boundry of the confidance interval of the accuracy : 0.8617061338856966
The upper boundry of the confidance interval of the accuracy : 0.9709274944328874


In [32]:
rois = ["opIFG_L", "planumtemp_L", "aMTG_L", "pITG_L", "opIFG_R", "planumtemp_R", "aMTG_R", "pITG_R"] # modified to rois of interest
ROIs = regionsOfInterest(data_results["acc_test"], pvalue, moyenne, rois, label, x_correl)

      coefficient          seed                      labels
349      0.066622        aMTG_L               L S_front_sup
394      0.066252        aMTG_L      R G_pariet_inf-Angular
530      0.061393        pITG_L       R G_front_inf-Orbital
300      0.060217        aMTG_L  L G_and_S_transv_frontopol
794      0.059196  planumtemp_R      L S_interm_prim-Jensen
1056     0.058729        pITG_R     L G_oc-temp_lat-fusifor
453      0.057348        pITG_L     L G_cingul-Post-ventral
41       0.056306       opIFG_L            L Pole_occipital
783      0.055522  planumtemp_R               L S_calcarine
941      0.051270        aMTG_R               L S_front_sup
The mean accuracy : 0.916316814159292
The standard deviation of the accuracy : 0.027862591976324208
The lower boundry of the confidance interval of the accuracy : 0.8617061338856966
The upper boundry of the confidance interval of the accuracy : 0.9709274944328874


In [37]:
SEEDS = ["opIFG_L", "planumtemp_L", "aMTG_L", "pITG_L", "opIFG_R", "planumtemp_R", "aMTG_R", "pITG_R"]
img_tpl = "/data/brambati/dataset/HCP/derivatives/seed-to-voxel-nilearn/results_3D/{seed_name}/sub-{participant_id}/sub-{participant_id}_seed-{seed_name}_concatenate_voxelcorrelations.nii.gz"
atlas_dest = datasets.fetch_atlas_destrieux_2009()
masker = NiftiLabelsMasker(atlas_dest.maps)

data_concat = {}

for seed in SEEDS:
    for sub in tqdm(df["subject_label"]):
        img_path = img_tpl.format(seed_name=seed, participant_id=sub)
        data_concat[(sub, seed)] = img4d2vector(img_path, masker)

/data/brambati/local/venvs/lingualab_workstation/lib/python3.8/site-packages/nilearn/datasets/atlas.py:288: UserWarning: `legacy_format` will default to `False` in release 0.11. Dataset fetchers will then return pandas dataframes by default instead of recarrays.
  warnings.warn(_LEGACY_FORMAT_MSG)
100%|█████████████████████████████████████████████████████████████████████████████████| 570/570 [03:43<00:00,  2.56it/s]


In [40]:
study_dir = Path('/data/brambati/dataset/HCP/derivatives/training_sex_diff/')
results_path_all_seed = study_dir / "results" / "LinearSVC" / "models_iteration-10000_seeds-all_concatenate_LinearSVC.pkl"




results_concat = joblib.load(results_path_all_seed)

coefs_concat = []
for result in results_concat:
    coefs_concat.append(result["model"].coef_[0])

coefs_concat = np.array(coefs_concat)

data_results_concat = pd.DataFrame.from_dict(results_concat)
moyenne_concat, scoresZ_concat, pvalue_concat, pvalue_bonf_concat = bootstrap_scores(coefs_concat)
moyenne_concat.shape

/data/brambati/local/venvs/lingualab_workstation/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearSVC from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


(1184,)

In [41]:
x_correl_concat = []
for sub in tqdm(df["subject_label"]):
    line_temp = []
    for seed in SEEDS:
        line_temp.append(data_concat[(sub, seed)])
    x_correl_concat.append(np.concatenate(line_temp))
x_correl_concat = np.array(x_correl_concat)

100%|█████████████████████████████████████████████████████████████████████████████| 570/570 [00:00<00:00, 126743.00it/s]


In [42]:
x_correl_concat.shape

(570, 1184)

In [44]:
rois = ["opIFG_L", "planumtemp_L", "aMTG_L", "pITG_L", "opIFG_R", "planumtemp_R", "aMTG_R", "pITG_R"] # modified to rois of interest
run_1 = run(df, data_concat, ["opIFG_L", "planumtemp_L", "aMTG_L", "pITG_L", "opIFG_R", "planumtemp_R", "aMTG_R", "pITG_R"])
ROIs = regionsOfInterest(data_results_concat["acc_test"], pvalue_concat, moyenne_concat, rois, label, run_1)
# bestRegion(ROIs)

100%|█████████████████████████████████████████████████████████████████████████████| 570/570 [00:00<00:00, 534724.51it/s]

      coefficient          seed                    labels
959      0.076520        aMTG_R          L S_temporal_inf
453      0.075518        pITG_L   L G_cingul-Post-ventral
866      0.069383  planumtemp_R          R S_front_middle
598      0.068284       opIFG_R  L G_and_S_cingul-Mid-Ant
1014     0.068131        aMTG_R          R S_front_middle
900      0.060643        aMTG_R     L G_front_inf-Orbital
1012     0.060388        aMTG_R    R S_collat_transv_post
1002     0.057794        aMTG_R            R Lat_Fis-post
923      0.055611        aMTG_R   L G_temp_sup-Plan_tempo
983      0.055345        aMTG_R   R G_oc-temp_med-Lingual
The mean accuracy : 0.8756353982300885
The standard deviation of the accuracy : 0.032657349067499834
The lower boundry of the confidance interval of the accuracy : 0.8116269940577888
The upper boundry of the confidance interval of the accuracy : 0.9396438024023882


In [45]:
rois = ["opIFG_L", "planumtemp_L", "aMTG_L", "pITG_L", "opIFG_R", "planumtemp_R", "aMTG_R", "pITG_R"] # modified to rois of interest
run_1 = run(df, data_concat, ["opIFG_L", "planumtemp_L", "aMTG_L", "pITG_L", "opIFG_R", "planumtemp_R", "aMTG_R", "pITG_R"])
ROIs = regionsOfInterest(data_results_concat["acc_test"], pvalue_concat, moyenne_concat, rois, label, run_1)
# bestRegion(ROIs)

100%|█████████████████████████████████████████████████████████████████████████████| 570/570 [00:00<00:00, 518489.11it/s]

      coefficient          seed                    labels
959      0.076314        aMTG_R          L S_temporal_inf
453      0.075315        pITG_L   L G_cingul-Post-ventral
866      0.069196  planumtemp_R          R S_front_middle
598      0.068101       opIFG_R  L G_and_S_cingul-Mid-Ant
1014     0.067948        aMTG_R          R S_front_middle
900      0.060479        aMTG_R     L G_front_inf-Orbital
1012     0.060226        aMTG_R    R S_collat_transv_post
1002     0.057638        aMTG_R            R Lat_Fis-post
923      0.055462        aMTG_R   L G_temp_sup-Plan_tempo
983      0.055196        aMTG_R   R G_oc-temp_med-Lingual
The mean accuracy : 0.8756353982300885
The standard deviation of the accuracy : 0.032657349067499834
The lower boundry of the confidance interval of the accuracy : 0.8116269940577888
The upper boundry of the confidance interval of the accuracy : 0.9396438024023882


In [49]:
rois = ["opIFG_L", "planumtemp_L", "aMTG_L", "pITG_L", "opIFG_R", "planumtemp_R", "aMTG_R", "pITG_R"] # modified to rois of interest
ROIs = regionsOfInterest(data_results_concat["acc_test"], pvalue_concat, moyenne_concat, rois, label, x_correl_concat)

      coefficient          seed                    labels
959      0.076649        aMTG_R          L S_temporal_inf
453      0.075645        pITG_L   L G_cingul-Post-ventral
866      0.069499  planumtemp_R          R S_front_middle
598      0.068399       opIFG_R  L G_and_S_cingul-Mid-Ant
1014     0.068246        aMTG_R          R S_front_middle
900      0.060744        aMTG_R     L G_front_inf-Orbital
1012     0.060490        aMTG_R    R S_collat_transv_post
1002     0.057891        aMTG_R            R Lat_Fis-post
923      0.055705        aMTG_R   L G_temp_sup-Plan_tempo
983      0.055438        aMTG_R   R G_oc-temp_med-Lingual
The mean accuracy : 0.8756353982300885
The standard deviation of the accuracy : 0.032657349067499834
The lower boundry of the confidance interval of the accuracy : 0.8116269940577888
The upper boundry of the confidance interval of the accuracy : 0.9396438024023882
